In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
%matplotlib notebook

## User settings

In [ ]:
N = 200 # number of nodes
L = 1 # domain length
dt = 0.001 # time step
t0 = 0 # initial time instant

U = 1 # convection velocity

theta = 0 # value of theta specifying method in time

refresh_time = 1 # basically the inverse of the framerate, in MILLISECONDS

## Actual code

Mesh and stability check:

In [ ]:
x = np.linspace(0, L, num=N)
dx = L/(N-1) # grid resolution
print() # new line
print('CFL number: ', ??? ) # CFL umber (stab. of convection in time)
print() # new line

Calculation of matrix B for centered in space scheme:

In [ ]:
# calcolate coefficients for centered
BEC =      ???
BWC =      ???
BPC =      ???     

# calculate matrix B for centered
BC = ???

# make B periodic: last row takes info from second point
???


Calculation of matrix B for upwind scheme:

In [ ]:
# calculate coefficients for upwind
BEU = ???
BWU = ???
BPU = ???

# calculate matrix B for upwind
BU = ???
# make B periodic: last row takes info from first point
???

Function to apply boundary conditions:

In [ ]:
def apply_bc(M,v):
    # this takes a matrix and a vector and modifies first and last row
    # in order to account for boundary conditions

    # Modify M and v for periodic boundary conditions
    M = ???
    v = ???

    return M,v

Function that returns exact solution at given time:

In [ ]:
def get_exact():
    a = (x-U*t0)%L
    return 16*((a**2)*(L-a)**2)/(L**4)

Initial condition:

In [ ]:
# initialise exact solution
exact_sol = get_exact()

# centered
c_curr = get_exact() # initial condition
c_new = np.zeros_like(x) # temperature at next time step
b_c = np.zeros_like(x) # right hand side vector

# upwind
u_curr = get_exact() # initial condition
u_new = np.zeros_like(x) # temperature at next time step
b_u = np.zeros_like(x) # right hand side vector

# plot it
fig, ax = plt.subplots(figsize=(10,5))
c_plot = ax.plot(x, c_curr) # update me!
u_plot = ax.plot(x, u_curr) # update me!
exact_plot = ax.plot(x, exact_sol) # update me!
ax.set_title('t = '+str(round(t0, 3))) # update me!
ax.legend(['centered', 'upwind', 'exact'])

Matrices for time step:

In [ ]:
# Matrix of coefficients (multiplies T_{n+1})
AC = np.identity(N) - dt * theta * BC
AU = np.identity(N) - dt * theta * BU


# Matrix that returns know term
EC = np.identity(N) + dt * (1-theta) * BC
EU = np.identity(N) + dt * (1-theta) * BU

# A * t_new = E * t_curr

Time step function:

In [ ]:
def time_step(ii):

    global c_curr, c_new, u_curr, u_new, b_c, b_u, AC, AU, EC, EU, t0, exact_sol

    # get known term and apply bc
    b_c = ????   # b-vector for centered scheme
    b_u[:] = ????   # b-vector for upwind scheme
    AC,b_c = ????   # Apply periodic B.C. for A and b
    AU,b_u = ????   # Apply periodic B.C. for A and b

    # solve system
    c_new[:] = np.linalg.solve(AC,b_c)
    u_new[:] = np.linalg.solve(AU,b_u)

    # update
    t0 += dt
    buffer_pointer = c_curr
    c_curr = c_new
    c_new = buffer_pointer
    buffer_pointer = u_curr
    u_curr = u_new
    u_new = buffer_pointer
    exact_sol[:] = get_exact()

    # plot
    c_plot[0].set_ydata(c_curr)
    u_plot[0].set_ydata(u_curr)
    exact_plot[0].set_ydata(exact_sol)
    ax.set_title('t = '+str(round(t0, 3)))

    return c_plot, u_plot, exact_plot

Animation:

In [ ]:
animation = FuncAnimation(fig, func=time_step, interval=refresh_time)
plt.show()